In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from os import path
try:
    import urllib
except ImportError:
    from urlparse import urlparse
import pandas
# import sys
# !{sys.executable} -m pip install lxml
import lxml
import html5lib

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:10.0.1) Gecko/20100101 Firefox/10.0.1'}

r = requests.get('https://www.statista.com/statistics/227249/greatest-gap-between-rich-and-poor-by-us-state/', headers=headers)

root = BeautifulSoup(r.text, 'html.parser')
table = root.find("table").prettify()

income_list = pandas.read_html(str(table), converters={"tbody": str})
income_table = pandas.DataFrame(income_list[0])
income_table.columns = ['States', 'Gini Coefficient']

In [ ]:
crime = requests.get('https://www.statista.com/statistics/200445/reported-violent-crime-rate-in-the-us-states/', headers=headers)

crime_soup = BeautifulSoup(crime.text, 'html.parser')

# View the content and find the appropriate table
crime_table = crime_soup.find("table").prettify()

# Save the aforementioned table as a variable
crime_list = pandas.read_html(str(crime_table), converters={"tbody": str})
crime_list = crime_list[0]
crime_list.columns = ['States', 'Crime Rate']

data = crime_list.merge(income_table, how = "inner", left_on = "States", right_on = "States")

data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame
import seaborn as sns

# sns.lineplot(data=data)

# fig, ax1 = plt.subplots()
# ax2 = ax1.twinx()  
# sns.regplot(x='States', y='Gini Coefficient', data=data, order=2, ax=ax1)
# sns.regplot(x='States', y='Crime Rate', data=data, order=2, ax=ax2)
# ax2.legend(handles=[a.lines[0] for a in [ax1,ax2]], 
#            labels=["f", "g"])
# plt.show()


fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('States')
ax1.set_ylabel('Gini Coefficient', color=color)
ax1.plot(data['States'], data['Gini Coefficient'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Crime Rate', color=color)  # we already handled the x-label with ax1
ax2.plot(data['States'], data['Crime Rate'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

# data.plot(x="States", y=["Gini Coefficient", "Crime Rate"], kind="bar")

# plt.show()

In [ ]:
# Polynomial
gini_fit = np.polyfit(data.index, data['Gini Coefficient'], 1)
crime_fit = np.polyfit(data.index, data['Crime Rate'], 1)

# Scatter plot
color1 = 'tab:blue'
ax1 = data.plot(kind='scatter', x='States', y='Gini Coefficient', color=color1, alpha=0.5, figsize=(10, 7))
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()
color2 = 'tab:red'
data.plot(kind='scatter', x='States', y= 'Crime Rate', color=color2, alpha=0.5, figsize=(10, 7), ax=ax2)
ax2.tick_params(axis='y', labelcolor=color2)

# Regression lines
plt.plot(data.index, gini_fit[0] * data.index + gini_fit[1], color=color1, linewidth=2)
ax2.plot(data.index, crime_fit[0] * data.index + crime_fit[1], color=color2, linewidth=2)

# Regression equation
ax1.text(25, 0.5, 'y={:.2f}+{:.2f}*x'.format(gini_fit[1], gini_fit[0]), color=color1, size=12)
ax1.text(10, 0.44, 'y={:.2f}+{:.2f}*x'.format(crime_fit[1], crime_fit[0]), color=color2, size=12)

# legend, title
plt.title('Gini Coefficent vs. Crime Rate for each state')

In [ ]:
median_income = requests.get('https://www.statista.com/statistics/233170/median-household-income-in-the-united-states-by-state/', headers=headers)

income_soup = BeautifulSoup(median_income.text, 'html.parser')

# View the content and find the appropriate table
income_table = income_soup.find("table").prettify()

income_list = pandas.read_html(str(income_table), converters={"tbody": str})
income_list = income_list[0]
income_list.columns = ['States', 'Median Income']
# income_list

# data comparing crime rate and median income
data2 = crime_list.merge(income_list, how = "inner", left_on = "States", right_on = "States")
data2

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('States')
ax1.set_ylabel('Median Income', color=color)
ax1.plot(data2['States'], data2['Median Income'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Crime Rate', color=color)  # we already handled the x-label with ax1
ax2.plot(data2['States'], data2['Crime Rate'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# Polynomial
income_fit = np.polyfit(data2.index, data2['Median Income'], 1)
crime_fit = np.polyfit(data2.index, data2['Crime Rate'], 1)

# Scatter plot
color1 = 'tab:blue'
ax1 = data2.plot(kind='scatter', x='States', y='Median Income', color=color1, alpha=0.5, figsize=(10, 7))
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()
color2 = 'tab:red'
data2.plot(kind='scatter', x='States', y= 'Crime Rate', color=color2, alpha=0.5, figsize=(10, 7), ax=ax2)
ax2.tick_params(axis='y', labelcolor=color2)

# Regression lines
plt.plot(data2.index, income_fit[0] * data2.index + income_fit[1], color=color1, linewidth=2)
ax2.plot(data2.index, crime_fit[0] * data2.index + crime_fit[1], color=color2, linewidth=2)

# Regression equation
ax2.text(25, 70000, 'y={:.2f}+{:.2f}*x'.format(income_fit[1], income_fit[0]), color=color1, size=12)
ax2.text(10, 40000, 'y={:.2f}+{:.2f}*x'.format(crime_fit[1], crime_fit[0]), color=color2, size=12)

# legend, title
plt.title('Median Income vs. Crime Rate for each state')